#### Includes

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import altair as alt
from altair import datum
import pandas as pd
import time
import os.path as P
from pytorch_lightning.utilities.cloud_io import load as pl_load
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()
None

In [3]:
from sources import load_dataset, default_config, parse_config
from run_train import setup_model
from src.trainer import Binary2DSegmentation

In [4]:
from steered_cnn.utils.rotequivariance_toolbox import *
from steered_cnn.steered_conv.steerable_filters import radial_steerable_filter, plot_filter
from steered_cnn.steered_conv import SteerableKernelBase, SteeredConv2d

## Initialization

In [5]:
RUN_ID = '14/5c19cfd8194b470dbc2d2a179f865365'

In [6]:
art_path = P.abspath(P.join('/home/gaby/mlflow/mlruns', RUN_ID, 'artifacts/'))
cfg = parse_config(art_path+'/cfg_extended.yaml')
cfg['script-arguments']['gpus']='0'

In [7]:
model = setup_model(cfg.model, 6, 1)
net = Binary2DSegmentation(model=model)
state_dict = pl_load(art_path+'/best-val-roc.ckpt'.format(25), map_location='cuda:0')['state_dict']
net.load_state_dict(state_dict)
net.eval().cuda()
None

## Weight Value Analysis

In [8]:
base = model.base
W = [_ for _ in state_dict.values() if (_.ndim==3 and _.shape[-1]==len(base.kernels_label))]
base.plot_weights_dist(W, scale_type='log', wrange=(1e-5,1))

alt.FacetChart(...)

In [9]:
base.plot_weights_dist(base.init_weights(100,500),  scale_type='log', wrange=(1e-3,1e-1))

alt.FacetChart(...)

## Gradient Analysis

In [10]:
trainD, validD, testD = load_dataset(cfg)

In [11]:
net.train()
net.zero_grad()
for batch in trainD:
    batch_cuda = {k: v.cuda() for k, v in batch.items()}
    loss = net.training_step(batch_cuda, 0)
    loss.backward()
    break

/home/gaby/.conda/envs/nnet/lib/python3.7/site-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/home/gaby/.conda/envs/nnet/lib/python3.7/site-packages/pytorch_lightning/core/lightning.py:418: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  "You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet."


In [13]:
weights_grad = torch.cat([m.weights.grad.flatten(end_dim=1) for m in net.modules() if isinstance(m, SteeredConv2d) and m.weights.shape[-1]==27])
base.plot_weights_dist(weights_grad, wrange=(1e-10, 1e-4))

alt.FacetChart(...)